In [1]:
import pandas as pd
import numpy as np
import pyodbc
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

In [2]:
svr_name = 'KU1ICDDWV011.kunet.ae'
db_name = 'CACM'
u_name = 'cacm_user'
u_pass = 'Welcome#123'

In [3]:
eng = create_engine("mssql+pyodbc://"+u_name+":"+u_pass+"@"+svr_name+"/"+db_name+"?driver=ODBC+Driver+17+for+SQL+Server")

In [4]:
query_employee = "select * from dbo.KU_SRC_EmployeeDetails"
query_payroll = "select * from ADERP.dbo.KU_Payroll_All"

In [5]:
employee = pd.read_sql(query_employee, eng)
payroll = pd.read_sql(query_payroll,eng)

In [6]:
employee_hire = employee[['EmpNo','HireDate']]

In [7]:
payroll['Pay Date'] = payroll['PayYear'].map(str)+'-'+payroll['PayMonth'].map(str)+'-'+str(26)

In [8]:
payroll = payroll.astype({'Pay Date':'datetime64'})

In [9]:
payroll_min = payroll.groupby('EmployeeNumber')['Pay Date'].min()

In [10]:
payroll_min = pd.DataFrame(payroll_min)

In [11]:
employee_hire = employee_hire.set_index('EmpNo')

In [12]:
employee_hire =employee_hire.reset_index()

In [13]:
payroll_min= payroll_min.reset_index()

In [14]:
employee_hire.columns

Index(['EmpNo', 'HireDate'], dtype='object')

In [15]:
payroll_min.columns

Index(['EmployeeNumber', 'Pay Date'], dtype='object')

In [16]:
output = employee_hire.merge(payroll_min,left_on='EmpNo',right_on='EmployeeNumber')

In [17]:
del(output['EmployeeNumber'])

In [21]:
output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1256 entries, 0 to 1255
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   EmpNo     1256 non-null   object        
 1   HireDate  1256 non-null   datetime64[ns]
 2   Pay Date  1256 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 39.2+ KB


In [22]:
output.head()

,EmpNo,HireDate,Pay Date
0,100043305,2016-01-10,2016-08-26
1,100049994,2018-11-13,2018-12-26
2,1816,2015-06-15,2018-01-26
3,1866,2015-12-07,2018-01-26
4,274,2008-12-14,2018-01-26


In [24]:
output['Diff'] = output['Pay Date'] < output['HireDate']

In [27]:
output['Diff']= output['Diff'].astype('int')

In [29]:
output[output['Diff'] == 1].to_excel('hire.xlsx')

In [9]:
payroll

,EmployeeNumber,PersonID,EmployeeGrade,PayYear,PayMonth,PayMonthNum,PayrollName,Basic,CoLA,MarketDiff,Vacation,ChildAllow,AdminStipend,ActingAllow,OtherLegacy,Gross
0,100020235,420048.0,SG12,2016,OCT,10,KU Monthly Payroll,6206.2,9930.07,0,517,0,0,0,0.0,16653.27
1,100020235,420048.0,SG12,2017,APR,4,KU Monthly Payroll,8463,13541,0,705,0,0,0,0.0,22709.00
2,100020242,308462.0,FA5,2017,APR,4,KU Monthly Payroll,6000,8000,0,0,0,0,0,0.0,14000.00
3,100020242,308462.0,FA5,2017,JUN,6,KU Monthly Payroll,6000,8000,0,0,0,0,0,0.0,14000.00
4,100020283,420260.0,FA5,2017,JAN,1,Researcher Monthly Payroll,6000,8000,0,0,0,0,0,0.0,14000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76973,PI4575,636005.0,1 Student,2018,JUN,6,Researcher Monthly Payroll,6600,4400,0,0,0,0,0,0.0,11000.00
76974,PI4577,629931.0,1 Student,2018,AUG,8,Researcher Monthly Payroll,2400,1600,0,0,0,0,0,0.0,4000.00
76975,PI4579,636012.0,1 Student,2018,MAY,5,Researcher Monthly Payroll,2400,1600,0,0,0,0,0,0.0,4000.00
76976,PI4580,635964.0,1 Student,2019,OCT,10,Researcher Monthly Payroll,0,0,0,0,0,0,0,0.0,0.00


In [10]:
employee.to_excel('employee_details.xlsx')

In [11]:
payroll.to_excel('payroll_all.xlsx')